# Artificial Intelligence - Fall 202 - Laboratory 10

## _First Order Predicate Logic:   Semantic Networks_

## Introduction

The purpose of this laboratory is to understand the concepts related to semantic networks and to implement simple mechanisms for determining the attributes of objects in a semantic network.

### Classes, attributes, objects

We consider that any **object** can belong to a **class**. This association is described by the relation $ Object \xrightarrow{isa} Class $. We will represent this relation by an instance of the predicate $ isa $, in the form $ isa (Object, Class) $.

One class can **inherit** another class (it is a sub-type). This relationship is called A-Kind-Of and is described as $ Subclass \xrightarrow{ako} Class $. We will represent this relation by an instance of the predicate $ ako $, in the form $ ako (Subclass, Class) $.

ako - a kind of

A class can have **attributes** that have **values**. We represent that a class $ Class $ has the attribute $ Atr $ with the value $ val $ as an atom $ Atr (Class, value) $. The same will be for an object.

Attributes are inherited from a class to a subclass of it and from the class to the object (instance). To determine what the value of an attribute might be for a particular object, we need to go up (to superclasses) on the $ isa $ and $ ako $ relations to find the first (closest) definition of the value of that attribute, in an ancestor of the object class.

We will represent a semantic network as a knowledge base containing exclusively atoms. The predicates will be $ isa $, $ ako $, and the predicates corresponding to the various attributes.

## Knowledge base


### Task 1

Model the knoledge base from the given example.


In [65]:
from FOP_helpful_funcs import *
def create_company(name,atr,original=False):
    # Fucntion to create the atoms asociated with a company
    control,location=atr

    isa=make_atom('ISA',(make_const(name),make_var('Company')))
    return (isa,
    make_atom('Controled',(make_const(name),make_const(control))),
    make_atom('Located',(make_const(name),make_const(location))))

def create_shop(name,mention,company_atr):
    # Fucntion to create the atoms asociated with a shop
    control,location=company_atr
    
    return (make_atom('ISA',(make_const(name),make_var('Shop'))),
    make_atom('Mentioned',(make_const(name),make_const(mention))),
    make_atom('Controled',(make_const(name),make_const(control))),
    make_atom('Located',(make_const(name),make_const(location))))
    

def create_region(name,type):
    # Fucntion to create the atoms asociated with a region
    return (make_atom('ISA',(make_const(name),make_var('Region'))),
    make_atom('Type',(make_const(name),make_var(type))))

def create_subregion(name,subregion):
    # Fucntion to create the atoms asociated with a subreagion
    isa=make_atom('ISA',(make_const(subregion),make_var('Subregion')))
    return (isa,
    make_atom('Subregion',(make_const(name),make_const(subregion))),)

        

    

KB = [
    # Structural KB

    #AKO
    make_atom('AKO',(make_var('Shop'),make_var('Company'))),
    make_atom('AKO',(make_var('Subregion'),make_var('Region'))),
    make_atom('AKO',(make_var('Location'),make_var('Subregion'))),
    #Attributes
    make_atom('Mentioned',(make_var('Shop'),make_var('value'))),
    make_atom('Controled',(make_var('Company'),make_var('value'))),
    make_atom('Located',(make_var('Company'),make_var('value'))),
    make_atom('Subreagion',(make_var('Region'),make_var('value'))),
    make_atom('Subreagion',(make_var('Subregion'),make_var('value'))),

]
    # Shops
for _ in create_shop('MY LOCAL CAFE','NEWS',['GLOBAL INVESMENT INC.','NEW YORK CITY']):
    KB.append(_)
    # Companies
for _ in create_company('GLOBAL INVESMENT INC.',['','WEST BAY']):
    KB.append(_)
for _ in create_company('BIG BUCKS CAFE',['BIG BUCKS COFFE','SEATLE']):
    KB.append(_)
    # Subregions
for _ in create_subregion('USA','WASHINGTON'):
    KB.append(_)
for _ in create_subregion('WASHINGTON','SEATLE'):
    KB.append(_)
for _ in create_subregion('USA','NEW YORK'):
    KB.append(_)
for _ in create_subregion('NEW YORK','NEW YORK CITY'):
    KB.append(_)
for _ in create_subregion('CAYMAN ISLAND','WEST BAY'):
    KB.append(_)
    # Region
for _ in create_region('USA','COUNTRY'):
    KB.append(_)
for _ in create_region('USA','OFFSHORE ZONE'):
    KB.append(_)

    

print("This is how the knowledge base is modeled: ", KB)

This is how the knowledge base is modeled:  [[4, 3, 'AKO', [([1, 'Shop', None], [1, 'Company', None])]], [4, 3, 'AKO', [([1, 'Subregion', None], [1, 'Region', None])]], [4, 3, 'AKO', [([1, 'Location', None], [1, 'Subregion', None])]], [4, 3, 'Mentioned', [([1, 'Shop', None], [1, 'value', None])]], [4, 3, 'Controled', [([1, 'Company', None], [1, 'value', None])]], [4, 3, 'Located', [([1, 'Company', None], [1, 'value', None])]], [4, 3, 'Subreagion', [([1, 'Region', None], [1, 'value', None])]], [4, 3, 'Subreagion', [([1, 'Subregion', None], [1, 'value', None])]], [4, 3, 'ISA', [([0, 'MY LOCAL CAFE'], [1, 'Shop', None])]], [4, 3, 'Mentioned', [([0, 'MY LOCAL CAFE'], [0, 'NEWS'])]], [4, 3, 'Controled', [([0, 'MY LOCAL CAFE'], [0, 'GLOBAL INVESMENT INC.'])]], [4, 3, 'Located', [([0, 'MY LOCAL CAFE'], [0, 'NEW YORK CITY'])]], [4, 3, 'ISA', [([0, 'GLOBAL INVESMENT INC.'], [1, 'Company', None])]], [4, 3, 'Controled', [([0, 'GLOBAL INVESMENT INC.'], [0, ''])]], [4, 3, 'Located', [([0, 'GLOBAL I

### Task 2

Implement the `infer_attr` function that determines the value inherited by a node for an attribute. Get one of the values that can be inherited.



In [70]:
# infer_attr finds the value of the given name attribute, inferred for the given name node. It is enough to find a value
# in any ancestor of the node (considering the condition that there is no other definition on the path between 
# the node and the ancestors of the attribute).
# It will return a tuple consisting of the value and the name of the node where the value was found
# Returns None if no value can be found
# If two different values are found, at a distance equal to the node, the message "CONTRADICTION" is returned instead.

def infer_attr(node_name, attribute_name, net):

    # Make a list with the ISA atoms asociated with the node
    L=[ k for k in KB if get_head(k)=='ISA' and get_args(k)[0][0][1]== node_name]
    # whle atoms left do:
    while len(L)!=0:
        # take the first
        l=L.pop(0)
        # If the head is the atribute name
        if get_head(l)==attribute_name:
            # make a list with the atoms with the head=> attribute name and class argument=> node name
            value=[ (attr,attribute_name) for attr in KB if get_head(attr)==attribute_name and get_args(attr)[0][0][1]==node_name]
            # if the result are more than one, contradiction
            if len(value)>=2:
                return 'CONTRADICTION'
                # if the result is unique , success
            elif len(value)==1:
                return(get_args(value[0][0])[0][1][1],attribute_name)
        
        # Get the class os the atom
        clas=get_args(l)[0][1]
        for k in KB:
            # Search in the KB for a AKO atom that link the current class with a superclass
            if get_head(k)=='AKO' and get_args(k)[0][0]==clas:
                superclas=get_args(k)[0][1]
                # If it is found, search the atributtes of the class and add to the current list
                atrib=[ atr for atr in KB if get_args(atr)[0][0]==superclas and get_head(atr) not in ['AKO','ISA']]
                L.extend(atrib)
    
    # If no result is found , FAIL
    return None



    
    
